In [1]:
%run env_setup.py
%matplotlib notebook
import importlib

In [72]:
import os
import torch
from torch import FloatTensor, nn
from torch.autograd import Variable
import torchvision
import yama
import yama.vision as yamavision
from tqdm import tqdm_notebook as tqdm

In [3]:
from yama.vision.datasets import LocalStorage, PaperSpaceGradientStorage

In [4]:
storage = LocalStorage(os.path.abspath('../../_data'))
#storage = PaperSpaceGradientStorage()

In [5]:
batch_size, image_size, noise_size = 256, (64, 64), 100

In [12]:
%env http_proxy=http://127.0.0.1:1087
%env https_proxy=http://127.0.0.1:1087

env: http_proxy=http://127.0.0.1:1087
env: https_proxy=http://127.0.0.1:1087


In [14]:
data_path = os.path.abspath('../../_data/cifar10')

data = torchvision.datasets.CIFAR10(root=data_path, download=True,
    transform=torchvision.transforms.Compose([
        torchvision.transforms.Resize(image_size),
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    ])
)

In [ ]:
#data_path = os.path.abspath('../../_data/lsun/')
#data_path = '../../_data/lsun'
data = yama.vision.datasets.LSUN(storage, classes=['bedroom_train'],
    transform=torchvision.transforms.Compose([
        torchvision.transforms.Resize(image_size),
        torchvision.transforms.CenterCrop(image_size),
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    ])
)

In [15]:
dataloader = torch.utils.data.DataLoader(data, batch_size, shuffle=True, num_workers=os.cpu_count())
n = len(dataloader); n

196

In [18]:
class DCGan_D(nn.Module):
    def __init__(self):
        pass
    
    def step(self, v, ):
        err = self(v)
        err.backward()

class DCGan_G(nn.Module):
    def __init__(self):
        pass

def train(D, G, opt_D, opt_G, epochs, loader, first=True):
    def make_trainable(m, b=True):
        for v in m.parameters(): v.require_grad = b
    
    n = len(loader)
    for ep in range(epochs):
        gen_iter = 0
        d_iter = 0
        d_iter_tgt = 0
        for real in tqdm(loader):
            if d_iter_tgt == 0 or d_iter >= d_iter_tgt:
                is_warm_up = first and gen_iter < 25:
                if is_warm_up or gen_iter % 500 == 0:
                    d_iter_tgt = 100
                else:
                    d_iter_tgt = 5
                d_iter = 0
            # TODO: process D
            d_iter += 1
            if d_iter == d_iter_tgt:
                # TODO: process G
                opt_G.step()
                gen_iter += 1
        